## 렌탈횟수가 30회 이상인 유저의 리스트
* customer, rental table

In [1]:
import MySQLdb
import pandas as pd

db = MySQLdb.connect(
    "db.fastcamp.us",
    "root",
    "dkstncks",
    "sakila",
    charset='utf8',
)

In [68]:
SQL_QUERY = """
    SELECT *
    FROM rental
"""

pd.read_sql(SQL_QUERY, db).head()


rental_id         rental_date  inventory_id  customer_id  \
0          1 2005-05-24 22:53:30           367          130   
1          2 2005-05-24 22:54:33          1525          459   
2          3 2005-05-24 23:03:39          1711          408   
3          4 2005-05-24 23:04:41          2452          333   
4          5 2005-05-24 23:05:21          2079          222   

          return_date  staff_id         last_update  
0 2005-05-26 22:04:30         1 2006-02-15 21:30:53  
1 2005-05-28 19:40:33         1 2006-02-15 21:30:53  
2 2005-06-01 22:12:39         1 2006-02-15 21:30:53  
3 2005-06-03 01:43:41         2 2006-02-15 21:30:53  
4 2005-06-02 04:33:21         1 2006-02-15 21:30:53

In [69]:
SQL_QUERY = """
    SELECT 
        customer_id,
        COUNT(rental_id)
           
    FROM rental
    GROUP BY 1
"""

pd.read_sql(SQL_QUERY, db).head()


customer_id  COUNT(rental_id)
0            1                32
1            2                27
2            3                26
3            4                22
4            5                38

** 1. WHERE 사용 & NEW SQL QUERY 사용 **

In [15]:
# 유저별로 몇번 렌탈 했을까? => 지표를 뽑아내자
SQL_QUERY = """
    SELECT 
        rental.customer_id,
        COUNT(rental.rental_id)          
    FROM rental  # FROM rental, customer 로하면 뻥튀기됨 아까 전에 상황과 같음
    GROUP BY 1
"""

pd.read_sql(SQL_QUERY, db)


customer_id  COUNT(rental.rental_id)
0              1                       32
1              2                       27
2              3                       26
3              4                       22
4              5                       38
5              6                       28
6              7                       33
7              8                       24
8              9                       23
9             10                       25
10            11                       24
11            12                       28
12            13                       27
13            14                       28
14            15                       32
15            16                       28
16            17                       21
17            18                       22
18            19                       24
19            20                       30
20            21                       35
21            22                       22
22            23                       30
23            24                       25
24            25                       29
25            26                       34
26            27                       31
27            28                       32
28            29                       36
29            30                       34
..           ...                      ...
569          570                       26
570          571                       24
571          572                       25
572          573                       29
573          574                       28
574          575                       29
575          576                       34
576          577                       27
577          578                       22
578          579                       27
579          580                       27
580          581                       27
581          582                       25
582          583                       23
583          584                       30
584          585                       24
585          586                       19
586          587                       26
587          588                       29
588          589                       28
589          590                       25
590          591                       27
591          592                       29
592          593                       26
593          594                       27
594          595                       30
595          596                       28
596          597                       25
597          598                       22
598          599                       19

[599 rows x 2 columns]

In [31]:
# 유저별로 몇번 렌탈 했을까? => 지표를 뽑아내자
# cutomer table과 매치시켜줘야함 => SQL을 다른 SQL에 조합가능
SQL_QUERY = """
    SELECT 
        rental.customer_id,
        COUNT(rental.customer_id) "total_count"          
    FROM rental
    GROUP BY 1
    ;
"""

NEW_SQL_QUERY = """
    SELECT customer.*, rental_per_customer.total_count
    FROM
        customer
        JOIN ({SQL_QUERY}) AS rental_per_customer            # <= 위 쿼리의 결과물을 새로운 테이블이라 생각함(이름지정)
            ON customer.customer_id = rental_per_customer.customer_id
    WHERE rental_per_customer.total_count >= 30
    ;
""".format(
    SQL_QUERY = SQL_QUERY.replace(";", ""),
)
# SQL_QUERY가 JOIN부분에 들어갈때 ;떄문에 끝났다고 인식하기때문에 오류가남
# 따라서 replace로 ;를 지워줘야함


#print(NEW_SQL_QUERY)


pd.read_sql(NEW_SQL_QUERY, db)


customer_id  store_id  first_name     last_name  \
0              1         1        MARY         SMITH   
1              5         1   ELIZABETH         BROWN   
2              7         1       MARIA        MILLER   
3             15         1       HELEN        HARRIS   
4             20         2      SHARON      ROBINSON   
5             21         1    MICHELLE         CLARK   
6             23         2       SARAH         LEWIS   
7             26         2     JESSICA          HALL   
8             27         2     SHIRLEY         ALLEN   
9             28         1     CYNTHIA         YOUNG   
10            29         2      ANGELA     HERNANDEZ   
11            30         1     MELISSA          KING   
12            35         2    VIRGINIA         GREEN   
13            38         1      MARTHA      GONZALEZ   
14            42         2     CAROLYN         PEREZ   
15            46         2   CATHERINE      CAMPBELL   
16            50         1       DIANE       COLLINS   
17            51         1       ALICE       STEWART   
18            53         1     HEATHER        MORRIS   
19            56         1      GLORIA          COOK   
20            64         2      JUDITH           COX   
21            66         2      JANICE          WARD   
22            71         1       KATHY         JAMES   
23            72         2     THERESA        WATSON   
24            75         2       TAMMY       SANDERS   
25            78         1        LORI          WOOD   
26            80         1     MARILYN          ROSS   
27            84         2        SARA         PERRY   
28            86         2  JACQUELINE          LONG   
29            87         1       WANDA     PATTERSON   
..           ...       ...         ...           ...   
148          452         1         TOM        MILNER   
149          454         2        ALEX       GRESHAM   
150          459         1       TOMMY       COLLAZO   
151          462         2      WARREN       SHERROD   
152          468         1         TIM          CARY   
153          469         2      WESLEY          BULL   
154          470         1      GORDON        ALLARD   
155          472         1        GREG        ROBINS   
156          473         2       JORGE      OLIVARES   
157          479         1     ZACHARY          HITE   
158          484         1     ROBERTO            VU   
159          494         2       RAMON        CHOATE   
160          499         2        MARC        OUTLAW   
161          502         1       BRETT      CORNWELL   
162          503         1       ANGEL       BARCLAY   
163          506         2      LESLIE        SEWARD   
164          513         2       DUANE         TUBBS   
165          520         2    MITCHELL  WESTMORELAND   
166          522         2      ARNOLD        HAVENS   
167          526         2        KARL          SEAL   
168          532         2        NEIL        RENNER   
169          533         1      JESSIE         MILAM   
170          535         1      JAVIER         ELROD   
171          550         2         GUY      BROWNLEE   
172          560         1      JORDAN     ARCHULETA   
173          566         1       CASEY          MENA   
174          569         2        DAVE      GARDINER   
175          576         2      MORRIS      MCCARTER   
176          584         2    SALVADOR          TEEL   
177          595         1    TERRENCE     GUNDERSON   

                                        email  address_id  active  \
0               MARY.SMITH@sakilacustomer.org           5       1   
1          ELIZABETH.BROWN@sakilacustomer.org           9       1   
2             MARIA.MILLER@sakilacustomer.org          11       1   
3             HELEN.HARRIS@sakilacustomer.org          19       1   
4          SHARON.ROBINSON@sakilacustomer.org          24       1   
5           MICHELLE.CLARK@sakilacustomer.org          25       1   
6              SARAH.LEWIS@sakilacustomer.org    

** 2. HAVING문으로 뽑기**

In [26]:
SQL_QUERY = """
    SELECT
        rental.customer_id,
        COUNT(rental.customer_id)
    FROM
        rental
    GROUP BY rental.customer_id
    HAVING COUNT(rental.customer_id) >= 30
    ;
"""

pd.read_sql(SQL_QUERY, db)

# 그룹바이를 하고나서야 카운트를 할수가 있는데 
# 그전에 WHERE를 적용하기 때문에 계산이 안됨
# WHERE -> HAVING 으로 해야함(연산이 끝난 상태에서 계산을 하는것임)

customer_id  COUNT(rental.customer_id)
0              1                         32
1              5                         38
2              7                         33
3             15                         32
4             20                         30
5             21                         35
6             23                         30
7             26                         34
8             27                         31
9             28                         32
10            29                         36
11            30                         34
12            35                         32
13            38                         34
14            42                         30
15            46                         34
16            50                         35
17            51                         33
18            53                         30
19            56                         30
20            64                         33
21            66                         34
22            71                         30
23            72                         30
24            75                         41
25            78                         31
26            80                         30
27            84                         33
28            86                         33
29            87                         30
..           ...                        ...
148          452                         32
149          454                         33
150          459                         38
151          462                         33
152          468                         39
153          469                         40
154          470                         32
155          472                         30
156          473                         34
157          479                         31
158          484                         30
159          494                         31
160          499                         30
161          502                         34
162          503                         32
163          506                         35
164          513                         31
165          520                         32
166          522                         33
167          526                         45
168          532                         32
169          533                         33
170          535                         32
171          550                         32
172          560                         30
173          566                         34
174          569                         32
175          576                         34
176          584                         30
177          595                         30

[178 rows x 2 columns]

** 3. pandas**
*  join -> merge
* where -> 부울리언

In [33]:
SQL_QUERY = """
    SELECT *
    FROM rental
    ;
"""

rental_df = pd.read_sql(SQL_QUERY, db)
rental_df.head()

rental_id         rental_date  inventory_id  customer_id  \
0          1 2005-05-24 22:53:30           367          130   
1          2 2005-05-24 22:54:33          1525          459   
2          3 2005-05-24 23:03:39          1711          408   
3          4 2005-05-24 23:04:41          2452          333   
4          5 2005-05-24 23:05:21          2079          222   

          return_date  staff_id         last_update  
0 2005-05-26 22:04:30         1 2006-02-15 21:30:53  
1 2005-05-28 19:40:33         1 2006-02-15 21:30:53  
2 2005-06-01 22:12:39         1 2006-02-15 21:30:53  
3 2005-06-03 01:43:41         2 2006-02-15 21:30:53  
4 2005-06-02 04:33:21         1 2006-02-15 21:30:53

In [35]:
SQL_QUERY = """
    SELECT *
    FROM customer
    ;
"""

customer_df = pd.read_sql(SQL_QUERY, db)
customer_df.head()

customer_id  store_id first_name last_name  \
0            1         1       MARY     SMITH   
1            2         1   PATRICIA   JOHNSON   
2            3         1      LINDA  WILLIAMS   
3            4         2    BARBARA     JONES   
4            5         1  ELIZABETH     BROWN   

                                 email  address_id  active  \
0        MARY.SMITH@sakilacustomer.org           5       1   
1  PATRICIA.JOHNSON@sakilacustomer.org           6       1   
2    LINDA.WILLIAMS@sakilacustomer.org           7       1   
3     BARBARA.JONES@sakilacustomer.org           8       1   
4   ELIZABETH.BROWN@sakilacustomer.org           9       1   

          create_date         last_update  
0 2006-02-14 22:04:36 2006-02-15 04:57:20  
1 2006-02-14 22:04:36 2006-02-15 04:57:20  
2 2006-02-14 22:04:36 2006-02-15 04:57:20  
3 2006-02-14 22:04:36 2006-02-15 04:57:20  
4 2006-02-14 22:04:36 2006-02-15 04:57:20

In [58]:
# 1. 그룹바이
import numpy as np
rental_per_customer_df = rental_df.groupby('customer_id').agg({'customer_id': np.size}) # 넘파이는 계산 어렵기때문에 dataframe으로 만들기위해 agg씀

In [59]:
rental_per_customer_df.rename(columns={"customer_id": "rental_count"}, inplace=True)

In [60]:
rental_per_customer_df["customer_id"] = rental_per_customer_df.index

In [62]:
customer_with_rental_count_df = pd.merge(customer_df, rental_per_customer_df, on='customer_id')

In [63]:
customer_with_rental_count_df

customer_id  store_id first_name  last_name  \
0              1         1       MARY      SMITH   
1              2         1   PATRICIA    JOHNSON   
2              3         1      LINDA   WILLIAMS   
3              4         2    BARBARA      JONES   
4              5         1  ELIZABETH      BROWN   
5              6         2   JENNIFER      DAVIS   
6              7         1      MARIA     MILLER   
7              8         2      SUSAN     WILSON   
8              9         2   MARGARET      MOORE   
9             10         1    DOROTHY     TAYLOR   
10            11         2       LISA   ANDERSON   
11            12         1      NANCY     THOMAS   
12            13         2      KAREN    JACKSON   
13            14         2      BETTY      WHITE   
14            15         1      HELEN     HARRIS   
15            16         2     SANDRA     MARTIN   
16            17         1      DONNA   THOMPSON   
17            18         2      CAROL     GARCIA   
18            19         1       RUTH   MARTINEZ   
19            20         2     SHARON   ROBINSON   
20            21         1   MICHELLE      CLARK   
21            22         1      LAURA  RODRIGUEZ   
22            23         2      SARAH      LEWIS   
23            24         2   KIMBERLY        LEE   
24            25         1    DEBORAH     WALKER   
25            26         2    JESSICA       HALL   
26            27         2    SHIRLEY      ALLEN   
27            28         1    CYNTHIA      YOUNG   
28            29         2     ANGELA  HERNANDEZ   
29            30         1    MELISSA       KING   
..           ...       ...        ...        ...   
569          570         2       IVAN   CROMWELL   
570          571         2    JOHNNIE   CHISHOLM   
571          572         1     SIDNEY   BURLESON   
572          573         1      BYRON        BOX   
573          574         2     JULIAN       VEST   
574          575         2      ISAAC    OGLESBY   
575          576         2     MORRIS   MCCARTER   
576          577         2    CLIFTON    MALCOLM   
577          578         2    WILLARD    LUMPKIN   
578          579         2      DARYL      LARUE   
579          580         1       ROSS       GREY   
580          581         1     VIRGIL    WOFFORD   
581          582         2       ANDY    VANHORN   
582          583         1   MARSHALL      THORN   
583          584         2   SALVADOR       TEEL   
584          585         1      PERRY   SWAFFORD   
585          586         1       KIRK    STCLAIR   
586          587         1     SERGIO  STANFIELD   
587          588         1     MARION     OCAMPO   
588          589         1      TRACY   HERRMANN   
589          590         2       SETH     HANNON   
590          591         1       KENT  ARSENAULT   
591          592         1   TERRANCE      ROUSH   
592          593         2       RENE  MCALISTER   
593          594         1    EDUARDO      HIATT   
594          595         1   TERRENCE  GUNDERSON   
595          596         1    ENRIQUE   FORSYTHE   
596          597         1    FREDDIE     DUGGAN   
597          598         1       WADE   DELVALLE   
598          599         2     AUSTIN    CINTRON   

                                     email  address_id  active  \
0            MARY.SMITH@sakilacustomer.org           5       1   
1      PATRICIA.JOHNSON@sakilacustomer.org           6       1   
2        LINDA.WILLIAMS@sakilacustomer.org           7       1   
3         BARBARA.JONES@sakilacustomer.org           8       1   
4       ELIZABETH.BROWN@sakilacustomer.org           9       1   
5        JENNIFER.DAVIS@sakilacustomer.org          10       1   
6          MARIA.MILLER@sakilacustomer.org          11       1   
7          SUSAN.WILSON@sakilacustomer.org          12       1   
8        MARGARET.MOORE@sakilacustomer.org          13       1   
9        DOROTHY.TAYLOR@sakilacustomer.org          14       1   
10        LISA.ANDERSON@sakilacustomer.org          15

In [66]:
customer_with_rental_count_df[customer_with_rental_count_df["rental_count"]>=30]

customer_id  store_id  first_name     last_name  \
0              1         1        MARY         SMITH   
4              5         1   ELIZABETH         BROWN   
6              7         1       MARIA        MILLER   
14            15         1       HELEN        HARRIS   
19            20         2      SHARON      ROBINSON   
20            21         1    MICHELLE         CLARK   
22            23         2       SARAH         LEWIS   
25            26         2     JESSICA          HALL   
26            27         2     SHIRLEY         ALLEN   
27            28         1     CYNTHIA         YOUNG   
28            29         2      ANGELA     HERNANDEZ   
29            30         1     MELISSA          KING   
34            35         2    VIRGINIA         GREEN   
37            38         1      MARTHA      GONZALEZ   
41            42         2     CAROLYN         PEREZ   
45            46         2   CATHERINE      CAMPBELL   
49            50         1       DIANE       COLLINS   
50            51         1       ALICE       STEWART   
52            53         1     HEATHER        MORRIS   
55            56         1      GLORIA          COOK   
63            64         2      JUDITH           COX   
65            66         2      JANICE          WARD   
70            71         1       KATHY         JAMES   
71            72         2     THERESA        WATSON   
74            75         2       TAMMY       SANDERS   
77            78         1        LORI          WOOD   
79            80         1     MARILYN          ROSS   
83            84         2        SARA         PERRY   
85            86         2  JACQUELINE          LONG   
86            87         1       WANDA     PATTERSON   
..           ...       ...         ...           ...   
451          452         1         TOM        MILNER   
453          454         2        ALEX       GRESHAM   
458          459         1       TOMMY       COLLAZO   
461          462         2      WARREN       SHERROD   
467          468         1         TIM          CARY   
468          469         2      WESLEY          BULL   
469          470         1      GORDON        ALLARD   
471          472         1        GREG        ROBINS   
472          473         2       JORGE      OLIVARES   
478          479         1     ZACHARY          HITE   
483          484         1     ROBERTO            VU   
493          494         2       RAMON        CHOATE   
498          499         2        MARC        OUTLAW   
501          502         1       BRETT      CORNWELL   
502          503         1       ANGEL       BARCLAY   
505          506         2      LESLIE        SEWARD   
512          513         2       DUANE         TUBBS   
519          520         2    MITCHELL  WESTMORELAND   
521          522         2      ARNOLD        HAVENS   
525          526         2        KARL          SEAL   
531          532         2        NEIL        RENNER   
532          533         1      JESSIE         MILAM   
534          535         1      JAVIER         ELROD   
549          550         2         GUY      BROWNLEE   
559          560         1      JORDAN     ARCHULETA   
565          566         1       CASEY          MENA   
568          569         2        DAVE      GARDINER   
575          576         2      MORRIS      MCCARTER   
583          584         2    SALVADOR          TEEL   
594          595         1    TERRENCE     GUNDERSON   

                                        email  address_id  active  \
0               MARY.SMITH@sakilacustomer.org           5       1   
4          ELIZABETH.BROWN@sakilacustomer.org           9       1   
6             MARIA.MILLER@sakilacustomer.org          11       1   
14            HELEN.HARRIS@sakilacustomer.org          19       1   
19         SHARON.ROBINSON@sakilacustomer.org          24       1   
20          MICHELLE.CLARK@sakilacustomer.org          25       1   
22             SARAH.LEWIS@sakilacustomer.org    